In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Folder_all_stuff')

In [ ]:
# load all library and packages

import os
import matplotlib.pyplot as plt
import cv2
import shutil
from PIL import Image
from string import Template
import numpy as np
import os
from google.colab.patches import cv2_imshow
import zipfile

In [ ]:
# with zipfile.ZipFile('video inputs/city_dark_frames.zip', 'r') as zip_ref:
#    zip_ref.extractall('city_dark_frames')

# Video processing methods
including 
(1) split the video into frames; 
(2) stitch frames back to video

In [ ]:
# video to frames
def video2imageFolder(input_file, output_path):
    '''
    Extracts the frames from an input video file
    and saves them as separate frames in an output directory.
    Input:
        input_file: Input video file.
        output_path: Output directorys.
    Output:
        None
    '''

    cap = cv2.VideoCapture()
    cap.open(input_file)

    if not cap.isOpened():
        print("Failed to open input video")

    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    frame_idx = 0

    while frame_idx < frame_count:
        ret, frame = cap.read()

        if not ret:
            print ("Failed to get the frame {}".format(frame_idx))
            continue

        out_name = os.path.join(output_path, 'f{:04d}.jpg'.format(frame_idx+1))
        ret = cv2.imwrite(out_name, frame)
        if not ret:
            print ("Failed to write the frame {}".format(frame_idx))
            continue

        frame_idx += 1
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)

# Image pre-processing and stitch methods

In [ ]:
def break_up_panorama(image_path, folder_name, frame_size, overlap_size):
    image = Image.open(image_path)
    width, height = image.size
    filename = os.path.basename(image_path)
    image_name = os.path.splitext(filename)[0]

    rows = height // (frame_size - overlap_size) + 1
    cols = width // (frame_size - overlap_size) + 1

    # create folder for split images
    os.makedirs(folder_name, exist_ok=True)
    print(folder_name)

    # loop over rows and columns and save each frame_size x frame_size image
    for i in range(rows):
        for j in range(cols):
            y_min = i * (frame_size - overlap_size)
            y_max = min(y_min + frame_size, height)
            x_min = j * (frame_size - overlap_size)
            x_max = min(x_min + frame_size, width)

            split_img = image.crop((x_min, y_min, x_max, y_max))

            filename = f"panorama_{i}_{j}.jpg"
            filepath = os.path.join(folder_name, filename)
            split_img.save(filepath)


def stitch_panorama(image_path, folder_name, reconstructed_path, frame_size, overlap_size):
    # Feed this function the same parameters as in 'break_up_panorama'

    original_image = Image.open(image_path)

    filename = os.path.basename(image_path)
    image_name = os.path.splitext(filename)[0]
    # folder_name = f"{image_name}_split_panorama"

    # calculate the number of rows and columns
    width, height = original_image.size
    rows = height // (frame_size - overlap_size) + 1
    cols = width // (frame_size - overlap_size) + 1

    # create a new empty image with the same dimensions as the original
    stitched_image = Image.new('RGB', (width, height))

    # loop over the split images and paste them onto the new image
    for i in range(rows):
        for j in range(cols):
            # load the split image
            # filename = f"panorama_{i}_{j}.jpg"
            filename = f"panorama_{i}_{j}_fake_B.png"
            filepath = os.path.join(folder_name, filename)
            split_image = Image.open(filepath)

            # calculate the paste location
            y_min = i * (frame_size - overlap_size)
            y_max = min(y_min + frame_size, height)
            x_min = j * (frame_size - overlap_size)
            x_max = min(x_min + frame_size, width)

            # paste the split image onto the new image
            stitched_image.paste(split_image, (x_min, y_min))

    # save the final stitched image to disk
    stitched_image.save(reconstructed_path + f"{image_name}_split_panorama.jpg")

In [ ]:
# frames to video
def imageFolder2video(input_path, output_path, video_name, fps):
      # create output folder if it doesn't exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    images = [img for img in os.listdir(input_path) if (img.endswith(".png") or img.endswith(".jpg"))]
    images.sort()
 
    frame = cv2.imread(os.path.join(input_path, images[0]))
    height, width, channels = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(os.path.join(output_path, video_name), fourcc, fps, (width, height))

    for image in images:
        img_path = os.path.join(input_path, image)
        frame = cv2.imread(img_path)

        if frame is None:
            print ("Failed to read the image {}".format(img_path))
            continue

        video.write(frame)

    video.release()

# **The following functions are utilized to process each video frame:**
The pipeline of video frame processing including:

1.   Histogram matching is applied to adjust the pixel values of each frame to a target histogram. This improves the contrast and brightness of the frames.
2.   Non-local means filtering is used to reduce noise while preserving edges and textures in the frames.
3.   Laplacian blending is utilized along the left and right edges of adjacent frames to smoothly transition between them. This helps to prevent visual artifacts that may appear when frames are merged together.

In [ ]:
### histogram mapping method
def hist_match(source_path, template_path):
  """
  Adjust the pixel values of a grayscale image such that its histogram
  matches that of a target image
  
  source: input image to be transformed
  template: target image with the desired histogram
  """
  source = cv2.imread(source_path)
  template = cv2.imread(template_path)

  old_shape = source.shape
  source = source.ravel()
  template = template.ravel()
  
  # get the set of unique pixel values and their corresponding indices and counts
  s_values, s_idx, s_counts = np.unique(source, return_inverse=True, return_counts=True)
  t_values, t_idx, t_counts = np.unique(template, return_inverse=True, return_counts=True)

  # calculate the normalized cumulative distribution functions for the two images
  s_quantiles = np.cumsum(s_counts).astype(np.float64)
  s_quantiles /= s_quantiles[-1]
  t_quantiles = np.cumsum(t_counts).astype(np.float64)
  t_quantiles /= t_quantiles[-1]

  # use linear interpolation of the quantiles to find the pixel values in the source image
  # that correspond to the pixel values in the template image
  interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

  return interp_t_values[s_idx].reshape(old_shape)

In [ ]:
def hist_match_handler(frames_folder_path, output_folder_path):
  # Create output folder if it does not exist
  if not os.path.exists(output_folder_path):
      os.makedirs(output_folder_path)

  file_list = sorted([os.path.join(frames_folder_path, f) for f in os.listdir(frames_folder_path) if f.endswith(('.png', '.jpg'))])

  # write the first image into the new folder
  file_ = os.path.join(output_folder_path, 'f0001_hist.png')
  cv2.imwrite(file_, cv2.imread(file_list[0]))

  # it takes the first frame as template for hist_match
  for i in range(1,len(file_list)):
      file_ = os.path.join(output_folder_path, 'f{:04d}_hist.png'.format(i+1))
      cv2.imwrite(file_, hist_match(file_list[i], file_list[0]))

In [ ]:
# to be used with the denoising method in the next cell
def fastNlMeansDenoisingColored(image, out_array, h, h, search_window, block_size):
    # Convert image to float64 data type
    image = image.astype(np.float64)

    # Split image into color channels
    b, g, r = cv2.split(image)

    # Compute the size of the image
    rows, cols = image.shape[:2]

    # Compute padding size based on block_size
    pad_size = block_size // 2

    # Pad each color channel with zeros around the edges
    b_padded = np.pad(b, ((pad_size, pad_size), (pad_size, pad_size)), mode='constant', constant_values=0)
    g_padded = np.pad(g, ((pad_size, pad_size), (pad_size, pad_size)), mode='constant', constant_values=0)
    r_padded = np.pad(r, ((pad_size, pad_size), (pad_size, pad_size)), mode='constant', constant_values=0)

    # Compute weights for each pixel based on the color and distance similarity
    weights = np.zeros((rows, cols), dtype=np.float64)
    for i in range(pad_size, rows + pad_size):
        for j in range(pad_size, cols + pad_size):
            block = b_padded[i-pad_size:i+pad_size+1, j-pad_size:j+pad_size+1]
            block_flat = block.flatten()
            distances = np.sum((block_flat - b_padded)**2, axis=1)
            weights[i-pad_size, j-pad_size] = np.sum(np.exp(-distances / (h**2)))

    # Normalize weights so they sum to 1
    weights /= np.sum(weights)

    # Apply weights to each color channel
    b_filtered = np.zeros((rows, cols), dtype=np.float64)
    g_filtered = np.zeros((rows, cols), dtype=np.float64)
    r_filtered = np.zeros((rows, cols), dtype=np.float64)
    for i in range(rows):
        for j in range(cols):
            b_block = b_padded[i:i+block_size, j:j+block_size]
            g_block = g_padded[i:i+block_size, j:j+block_size]
            r_block = r_padded[i:i+block_size, j:j+block_size]
            b_filtered[i, j] = np.sum(b_block * weights[i:i+block_size, j:j+block_size])
            g_filtered[i, j] = np.sum(g_block * weights[i:i+block_size, j:j+block_size])
            r_filtered[i, j] = np.sum(r_block * weights[i:i+block_size, j:j+block_size])

    # Merge color channels back into image
    filtered_image = cv2.merge((b_filtered, g_filtered, r_filtered))

    # Convert image back to uint8 data type
    filtered_image = np.clip(filtered_image, 0, 255).astype(np.uint8)

    return filtered_image


In [ ]:
### non-local means filter denoise method
def denoise_frames(frames_folder_path, output_folder_path, h=5, 
                   search_window=15, block_size=5, temporal_window=3):
    # Create output folder if it does not exist
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # List all the frames in the input folder
    frames_list = sorted(os.listdir(frames_folder_path))

    # Loop through all the frames in the input folder
    for i in range(len(frames_list)):
        # Read the current frame image
        current_frame = cv2.imread(os.path.join(frames_folder_path, frames_list[i]))

        # Initialize an empty list to store neighboring frames
        neighbor_frames = []

        # Loop through the previous frames to get the neighboring frames
        for j in range(max(0, i - temporal_window), i):
            neighbor_frames.append(cv2.imread(os.path.join(frames_folder_path, frames_list[j])))

        # Loop through the next frames to get the neighboring frames
        for j in range(i + 1, min(i + temporal_window + 1, len(frames_list))):
            neighbor_frames.append(cv2.imread(os.path.join(frames_folder_path, frames_list[j])))

        # Apply non-local means filter to remove noise in the current frame
        denoised = fastNlMeansDenoisingColored(current_frame, None, h, h, search_window, block_size)

        # Initialize an empty list to store the denoised neighboring frames
        denoised_neighbor_frames = []

        # Apply non-local means filter to remove noise in the neighboring frames
        for neighbor_frame in neighbor_frames:
            denoised_neighbor = fastNlMeansDenoisingColored(neighbor_frame, None, h, h, search_window, block_size)

            # Add the denoised neighboring frames to a list
            denoised_neighbor_frames.append(denoised_neighbor)

        # Take the median of the denoised neighboring frames to get a single denoised neighboring frame
        if len(denoised_neighbor_frames) > 0:
            denoised_neighbor = np.median(denoised_neighbor_frames, axis=0)

            # Convert color format if needed
            #if denoised_neighbor.ndim == 3 and denoised_neighbor.shape[2] == 3:
            if denoised_neighbor.dtype == 'float64':
                denoised_neighbor = cv2.convertScaleAbs(denoised_neighbor)

            # Apply median blur to remove noise in the denoised neighboring frame
            denoised_neighbor = cv2.medianBlur(denoised_neighbor, 3)

            # Combine the denoised current frame and the denoised neighboring frame
            denoised = cv2.addWeighted(denoised, 0.5, denoised_neighbor, 0.5, 0)

        # Save the denoised image to output folder
        # cv2.imwrite(os.path.join(output_folder_path, frames_list[i]), denoised)
        output_filename = 'f{:04d}_denoise.png'.format(i+1)
        cv2.imwrite(os.path.join(output_folder_path, output_filename), denoised)

In [ ]:
### laplacian blending method on left/right edge
def laplacian_blend(image1, image2, overlap_size):
    # image 1 is left side, image 2 is right side, return image2blend
    
    # Transpose images to blend left edge (same logic as blending top edge)
    image1 = np.transpose(image1, (1, 0, 2))
    image2 = np.transpose(image2, (1, 0, 2))

    # Generate Gaussian and Laplacian pyramids for both images
    gaussian1 = image1.copy()
    gaussian2 = image2.copy()
    laplacian1 = [gaussian1]
    laplacian2 = [gaussian2]

    num_levels = int(np.floor(np.log2(min(gaussian1.shape[0], gaussian2.shape[0], overlap_size))) - 4)

    for i in range(num_levels):
        gaussian1_down = cv2.pyrDown(gaussian1)
        gaussian2_down = cv2.pyrDown(gaussian2)
        laplacian1.append(cv2.subtract(gaussian1, cv2.pyrUp(gaussian1_down)))
        laplacian2.append(cv2.subtract(gaussian2, cv2.pyrUp(gaussian2_down)))
        gaussian1 = gaussian1_down
        gaussian2 = gaussian2_down

    # Combine the Laplacian pyramids of the two images
    blended_pyramid = []
    for i in range(num_levels, 0, -1):
        size = (laplacian1[i - 1].shape[1], laplacian1[i - 1].shape[0])
        if i == num_levels:
            blended_pyramid.append(cv2.addWeighted(laplacian1[i - 1], 0.5, laplacian2[i - 1], 0.5, 0))
        else:
            overlap_region1 = blended_pyramid[-1][:, -overlap_size:]
            overlap_region2 = laplacian2[i - 1][:, :overlap_size]
            alpha = np.linspace(0, 1, overlap_size)
            overlap_region = cv2.addWeighted(overlap_region1, 1 - alpha, overlap_region2, alpha, 0)
            blended_pyramid.append(np.concatenate((laplacian1[i - 1][:, :-overlap_size], overlap_region), axis=1))

    # Combine the non-overlapping regions of the two images
    blended_image = blended_pyramid[-1]
    for i in range(num_levels - 1, -1, -1):
        blended_image = cv2.pyrUp(blended_image)
        if i == num_levels - 1:
            blended_image = cv2.addWeighted(laplacian1[i], 0.5, laplacian2[i], 0.5, 0)
        else:
            overlap_region1 = blended_image[:, -overlap_size:]
            overlap_region2 = blended_pyramid[i][:, :overlap_size]
            alpha = np.linspace(0, 1, overlap_size)
            overlap_region = cv2.addWeighted(overlap_region1, 1 - alpha, overlap_region2, alpha, 0)
            blended_image = np.concatenate((blended_pyramid[i][:, :-overlap_size], overlap_region), axis=1)
            blended_image = cv2.addWeighted(laplacian1[i], 0.5, blended_image, 0.5, 0)

    # Crop the overlapping region
    blended_image = np.transpose(blended_image, (1, 0, 2))
    
    return blended_image

In [ ]:
def laplacian_blend_handler(folder_path, overlap_size, output_folder_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png','.jpg'))])
    num_images = len(image_files)
    if num_images < 2:
        print("Error: folder must contain at least two image files")
        return

    # create output folder if it doesn't exist
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    for i in range(num_images - 1):
        # extract image order numbers from file names
        image1_order = int(image_files[i][1:5])
        image2_order = int(image_files[i+1][1:5])
        output_filename = 'f{:04d}_blend.png'.format(image2_order)

        # load images and blend them
        image1 = cv2.imread(os.path.join(folder_path, image_files[i]))
        image2 = cv2.imread(os.path.join(folder_path, image_files[i+1]))
        blended_image = laplacian_blend(image1, image2, overlap_size)

        # save blended image to output folder
        cv2.imwrite(os.path.join(output_folder_path, output_filename), blended_image)

In [ ]:
### homography method
# but we don't use this method in our final version after testing

def homography(original_pic, transformed_pic):

  # Load images
  img_a = original_pic
  img_b = transformed_pic

  # Detect and extract keypoints from both images
  orb = cv2.ORB_create(nfeatures=1000)
  kp_a, des_a = orb.detectAndCompute(img_a, None)
  kp_b, des_b = orb.detectAndCompute(img_b, None)

  # Match keypoints using brute-force matcher
  bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
  matches = bf.match(des_a, des_b)

  # Sort matches by distance
  matches = sorted(matches, key=lambda x: x.distance)

  # Get the top N matches
  n_matches = 50
  matches = matches[:n_matches]

  # Get corresponding keypoints from both images
  pts_a = np.float32([kp_a[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
  pts_b = np.float32([kp_b[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

  # Compute homography matrix
  homography, _ = cv2.findHomography(pts_b, pts_a, cv2.RANSAC)

  # Apply homography to image B
  img_b_aligned = cv2.warpPerspective(img_b, homography, (img_a.shape[1], img_a.shape[0]))

  # Display result
  cv2_imshow(img_a)
  cv2_imshow(img_b)
  cv2_imshow(img_b_aligned)
  cv2.waitKey(0)
  cv2.destroyAllWindows()


# **Apply method on the 'city' video**

In [ ]:
# some pro-processing
# cropping video to square for better output
# load input video
cap = cv2.VideoCapture('video inputs/city_water.mp4')

# get video dimensions
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# calculate square dimensions
dim = min(width, height)

# calculate top-left corner coordinates of the square
x = int((width - dim) / 2)
y = int((height - dim) / 2)

# create output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('city_water_centered.mp4', fourcc, 30.0, (dim, dim))

# loop through frames and reframe each frame as a square
while cap.isOpened():
     ret, frame = cap.read()
     if not ret:
         break

     # crop frame to square dimensions
     frame = frame[y:y+dim, x:x+dim]

     # resize frame to output dimensions
     frame = cv2.resize(frame, (dim, dim))

     # write frame to output video
     out.write(frame)

     # display output frame
     cv2_imshow(frame)
     if cv2.waitKey(1) & 0xFF == ord('q'):
         break

# release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# move all fake images to another filder
# Create the directory to move the files to
# if not os.path.exists('fake_images'):
#     os.mkdir('fake_images')
# # directory = 'results/monet2photo_pretrained/test_latest/images'
# directory = 'frames/monet'

# # Loop over all files in the current directory
# for filename in os.listdir(directory):
#     # Check if the filename contains the string 'fake'
#     print(filename)
#     if 'fake' in filename:
#         # Create the full file path
#         file_path = os.path.join(directory, filename)
#         # Check if the file exists in the destination directory
#         if not os.path.exists(os.path.join('fake_images', filename)):
#             # Move the file to the 'fake_images' directory
#             shutil.move(file_path, 'fake_images')
#             print(f"Moved {filename} to fake_images directory")
#         else:
#             print(f"{filename} already exists in fake_images directory")

# **First application: apply Pix2Pix model**

In [ ]:
# make video from synthesized images w/o any image processing
imageFolder2video(input_path='city_dark_frames/dark_images', 
                  output_path='city_dark_frames/fake_video', 
                  video_name='fake_video.mp4', 
                  fps=30)

In [ ]:
# Histogram matching
hist_match_handler(frames_folder_path='city_dark_frames/dark_images', 
                   output_folder_path='city_dark_frames/hist_output')

In [ ]:
# Denoising
denoise_frames(frames_folder_path='city_dark_frames/dark_images',
               output_folder_path='city_dark_frames/denoise_output',
               h=5, search_window=15, block_size=5, temporal_window=3)

In [ ]:
# Laplacian blending
laplacian_blend_handler(folder_path='city_dark_frames/dark_images', 
                        overlap_size=50, 
                        output_folder_path='city_dark_frames/laplacian_output')

In [ ]:
# Sequentially apply these 3 methods by the order of histogram matching --> Denoising --> Laplacian Blending
# histogram matching frame alreay generated, we use these output as input of denoising to proceed

# Denoising after Histogram matching
denoise_frames(frames_folder_path='city_dark_frames/hist_output',
               output_folder_path='city_dark_frames/hist_denoise_output',
               h=5, search_window=15, block_size=5, temporal_window=3)

In [ ]:
# Laplacian blending after denosing
laplacian_blend_handler(folder_path='city_dark_frames/hist_denoise_output', 
                        overlap_size=50, 
                        output_folder_path='city_dark_frames/hist_denoise_laplacian_output')

In [ ]:
# make final video from frames w image processing
imageFolder2video(input_path='city_dark_frames/hist_denoise_laplacian_output', 
                  output_path='city_dark_frames/final_video', 
                  video_name='final_video.mp4',
                  fps=30)

# **Second application: apply cycleGAN model**

In [ ]:
# make video from synthesized images w/o any image processing
imageFolder2video(input_path='vangogh_frames/vangogh_output', 
                  output_path='vangogh_frames/vangogh_synthesized_video', 
                  video_name='vangogh_synthesized_video.mp4', 
                  fps=30)

In [ ]:
# Histogram matching
hist_match_handler(frames_folder_path='vangogh_frames/vangogh_output',
                   output_folder_path='vangogh_frames/hist_output')

In [ ]:
# Denoising
denoise_frames(frames_folder_path='vangogh_frames/vangogh_output',
               output_folder_path='vangogh_frames/denoise_output',
               h=5, search_window=15, block_size=5, temporal_window=3)

In [ ]:
# Laplacian blending
laplacian_blend_handler(folder_path='vangogh_frames/vangogh_output', 
                        overlap_size=50, 
                        output_folder_path='vangogh_frames/laplacian_output')

In [ ]:
# # Sequentially apply these 3 methods by the order of histogram matching --> Denoising --> Laplacian Blending
# histogram matching frame alreay generated, we use these output as input of denoising to proceed

# Denoising after Histogram matching
denoise_frames(frames_folder_path='vangogh_frames/hist_output',
               output_folder_path='vangogh_frames/hist_denoise_output',
               h=5, search_window=15, block_size=5, temporal_window=3)

# Laplacian blending after denosing
laplacian_blend_handler(folder_path='vangogh_frames/hist_denoise_output', 
                        overlap_size=50, 
                        output_folder_path='vangogh_frames/hist_denoise_laplacian_output')

In [ ]:
# make final video from frames w image processing
imageFolder2video(input_path='vangogh_frames/hist_denoise_laplacian_output', 
                  output_path='vangogh_frames/final_video', 
                  video_name='final_video.mp4',
                  fps=30)